In [113]:
#imports
import re

# Task 1 - Table Driven Lexer

In [114]:
class Token:
    def __init__(self, inputValue, inputType):
        self.value = inputValue
        self.type = inputType

class Lexer:
    def __init__(self, input):
        self.code = input
        self.tokenList = []
        self.pos = 0
        self.tokenTable = [
            (r"not|let|return|if|else|for|while|fun|->", "KEYWORDS"),
            (r"[\,\(\)\-\=\:\;\{\}]", "OTHER"),

            (r"\b(float|int|bool|colour)\b", "TYPE"),
            
            (r"\b(true|false)\b", "BOOLEANLITERAL"),
            (r"\d+\.\d+", "FLOATLITERAL"),
            (r"\d+", "INTEGERLITERAL"),
            (r"\#[0-9a-fA-F]{6}", "COLOURLITERAL"),

            (r"\b(__width)\b", "PADWIDTH"),
            (r"\b(__height)\b", "PADHEIGHT"),

            (r"\b(__read)\b", "__READ"),
            (r"\b(__randi)\b", "__RANDI"),

            (r"[\*/]|and|AND", "MULTIPLICATIVEOP"),
            (r"[\+]|or|OR", "ADDITIVEOP"),
            (r"==|!=|<=|>=|[\<\>]", "RELATIONALOP"),

            (r"[a-zA-Z]([a-zA-Z0-9]|_)*", "IDENTIFIER"),

            (r"\b(__print)\b", "__PRINT"),
            (r"\b(__delay)\b", "__DELAY"),
            (r"\b(__pixelr)\b", "__PIXELR"),
            (r"\b(__pixel)\b", "__PIXEL"),
        ]
        self.scanText()
        
        print("L E X E R")
        print("Code accepted by Lexer")
        # print("Tokens: " + str(self.tokenList) +"\n")
  
    def scanText(self):
        # loops while less than size of string
        while self.pos < len(self.code):
            # ignores whitespace
            if self.code[self.pos] == " " or self.code[self.pos] == "\n":
                self.pos += 1
                
            # if not whitespace
            else:
                value, type = None, None

                # goes through table of patterns
                for pattern, tokenType in self.tokenTable:
                    match = re.match(pattern, self.code[self.pos:])

                    # if they match
                    if match is not None:
                        type = tokenType
                        value = match.group()
                        break
                
                # if value and type remained empyty
                if value == None or type == None:
                    raise Exception("Error in Syntax")
                # if both value and type have been assigned
                else:
                    if type == "KEYWORDS" or type == "OTHER":
                        self.tokenList.append((value.upper(), value))
                        # self.tokenList.append(Token(value, "<"+value+">"))
                    else:
                        self.tokenList.append((type, value))
                        # self.tokenList.append(Token(value, type))
                    self.pos += len(value)

In [115]:
f = open('code.txt', 'r'); code = f.read(); f.close()
lexer = Lexer(code)

L E X E R
Code accepted by Lexer


# Task 2 - Hand-crafted LL(k) parser

In [116]:
class TerminalNode:
    def __init__(self, name, value):
        self.name = name
        self.value = value


class NTerminalNode:
    def __init__(self, name):
        self.name = name
        self.children = []
        

class Parser:
    def __init__(self, input):
        self.lexer = Lexer(input)
        self.tree = None

        print("\nP A R S E R")
        self.parseCode()
        print("Code Accepted by Parser")        


    def parseCode(self):
        self.program()
        

    def getNextToken(self):
        self.lexer.tokenList.pop(0)


    def padRead(self, tree):
        nTree = NTerminalNode("PADREAD")

        if self.lexer.tokenList[0][0] == "__READ":
            nTree.children.append(TerminalNode("__READ", self.lexer.tokenList[0][1])); self.getNextToken()
            self.expr(nTree)

            if self.lexer.tokenList[0][0] == ",":
                self.getNextToken()
                self.expr(nTree)
                tree.children.append(nTree)            
            else:
                raise Exception ("Error: missing ','")


    def padRandI(self, tree):
        nTree = NTerminalNode("PADRANDI")

        if self.lexer.tokenList[0][0] == "__RANDI":
            nTree.children.append(TerminalNode("__RANDI", self.lexer.tokenList[0][1])); self.getNextToken()
            self.expr(nTree)
            tree.children.append(nTree)


    def literal(self, tree):
        nTree = NTerminalNode("LITERAL")

        nTree.children.append(TerminalNode(self.lexer.tokenList[0][0], self.lexer.tokenList[0][1])); self.getNextToken()
        tree.children.append(nTree)


    def actualParams(self, tree):
        nTree = NTerminalNode("ACTUALPARAMS")
        self.expr(nTree)

        while self.lexer.tokenList and self.lexer.tokenList[0][0] == ",":
            self.getNextToken()
            self.expr(nTree)
        
        tree.children.append(nTree)


    def functionCall(self, tree):
        nTree = NTerminalNode("FUNCTIONCALL")

        if self.lexer.tokenList[0][0] == "IDENTIFIER":
            nTree.children.append(TerminalNode("IDENTIFIER", self.lexer.tokenList[0][1])); self.getNextToken()
            
            if self.lexer.tokenList[0][0] == "(":
                self.getNextToken()
            
                if self.lexer.tokenList[0][0] != ")":
                    self.actualParams(nTree)
            
                    if self.lexer.tokenList[0][0] == ")":
                        self.getNextToken()
                        tree.children.append(nTree)
                        
                elif self.lexer.tokenList[0][0] == ")":
                    self.getNextToken()
                    tree.children.append(nTree)     

                else:
                    raise Exception ("Error: missing ')")
            else:
                raise Exception ("Error: missing '(")


    def subExpr(self, tree):
        nTree = NTerminalNode("SUBEXPR")

        if self.lexer.tokenList[0][0] == "(":
            self.getNextToken()
            self.expr(nTree)

            if self.lexer.tokenList[0][0] == ")":
                self.getNextToken()
                tree.children.append(nTree)
            
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")


    def unary(self, tree):
        nTree = NTerminalNode("UNARY")

        if self.lexer.tokenList[0][0] in ["-", "NOT"]:
            nTree.children.append(TerminalNode(self.lexer.tokenList[0][0], self.lexer.tokenList[0][1])); self.getNextToken()
            self.expr(nTree)
            tree.children.append(nTree)

        else:
            raise Exception ("Error in Parser")


    def factor(self, tree):
        nTree = NTerminalNode("FACTOR")

        # LITERAL
        if self.lexer.tokenList[0][0] in ["BOOLEANLITERAL", "INTEGERLITERAL", "FLOATLITERAL", "COLOURLITERAL"]:
            self.literal(nTree)

        # IDENTIFIER & FUNCTIONCALL
        elif self.lexer.tokenList[0][0] == "IDENTIFIER":   
            if len(self.lexer.tokenList) > 1:
                if self.lexer.tokenList[1][0] == "(":
                    self.functionCall(nTree)
                else:
                    nTree.children.append(TerminalNode("IDENTIFIER", self.lexer.tokenList[0][1])); self.getNextToken()
            else:
                nTree.children.append(TerminalNode("IDENTIFIER", self.lexer.tokenList[0][1])); self.getNextToken()

        # SUBEXPR
        elif self.lexer.tokenList[0][0] == "(":
            self.subExpr(nTree)

        # UNARY
        elif self.lexer.tokenList[0][0] == "-" or self.lexer.tokenList[0][0] == "NOT":
            self.unary(nTree)

        # PADRANDI
        elif self.lexer.tokenList[0][0] == "__RANDI":
            self.padRandI(nTree)

        # PADWIDTH
        elif self.lexer.tokenList[0][0] == "PADWIDTH":
            nTree.children.append(TerminalNode("PADWIDTH", self.lexer.tokenList[0][1])); self.getNextToken()

        # PADHEIGHT
        elif self.lexer.tokenList[0][0] == "PADHEIGHT":
            nTree.children.append(TerminalNode("PADHEIGHT", self.lexer.tokenList[0][1])); self.getNextToken()

        #PADREAD
        elif self.lexer.tokenList[0][0] == "__READ":
            self.padRead(nTree)

        else:
            raise Exception ("Error in Parser")
        
        tree.children.append(nTree)
    

    def term(self, tree):
        nTree = NTerminalNode("TERM")
        self.factor(nTree)
        
        while self.lexer.tokenList and self.lexer.tokenList[0][0] == "MULTIPLICATIVEOP":
            nTree.children.append(TerminalNode("MULTIPLICATIVEOP", self.lexer.tokenList[0][1])); self.getNextToken()
            self.factor(nTree)
        
        tree.children.append(nTree)
        

    def simpleExpr(self, tree):
        nTree = NTerminalNode("SIMPLEEXPR")
        self.term(nTree)
        
        while self.lexer.tokenList and self.lexer.tokenList[0][0] == "ADDITIVEOP" or self.lexer.tokenList[0][1] == "-":
            nTree.children.append(TerminalNode("ADDITIVEOP", self.lexer.tokenList[0][1])); self.getNextToken()
            self.term(nTree)
        
        tree.children.append(nTree)


    def expr(self, tree):
        nTree = NTerminalNode("EXPR")
        self.simpleExpr(nTree)
        
        while self.lexer.tokenList and self.lexer.tokenList[0][0] == "RELATIONALOP":
            nTree.children.append(TerminalNode("RELATIONALOP", self.lexer.tokenList[0][1])); self.getNextToken()
            self.simpleExpr(nTree)
            

        tree.children.append(nTree)


    def assignment(self, tree):
        nTree = NTerminalNode("ASSIGNMENT")

        if self.lexer.tokenList[0][0] == "IDENTIFIER":
            nTree.children.append(TerminalNode("IDENTIFIER", self.lexer.tokenList[0][1])); self.getNextToken()

            if self.lexer.tokenList[0][0] == "=":
                nTree.children.append(TerminalNode("EQUALS", self.lexer.tokenList[0][1])); self.getNextToken()
                self.expr(nTree)
                tree.children.append(nTree)
            
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")


    def variableDecl(self, tree):
        nTree = NTerminalNode("VARIABLEDECL")

        if self.lexer.tokenList[0][0] == "LET":
            self.getNextToken()
            # nTree.children.append(TerminalNode("LET", self.lexer.tokenList[0][1])); self.getNextToken()
            
            if self.lexer.tokenList[0][0] == "IDENTIFIER":
                nTree.children.append(TerminalNode("IDENTIFIER", self.lexer.tokenList[0][1])); self.getNextToken()
                
                if self.lexer.tokenList[0][0] == ":":
                    self.getNextToken()

                    if self.lexer.tokenList[0][0] == "TYPE":
                        nTree.children.append(TerminalNode("TYPE", self.lexer.tokenList[0][1])); self.getNextToken()

                        if self.lexer.tokenList[0][0] == "=":
                            nTree.children.append(TerminalNode("EQUALS", self.lexer.tokenList[0][1])); self.getNextToken()
                            self.expr(nTree)
                            tree.children.append(nTree)

                        else:
                            raise Exception ("Error in Parser")
                    else:
                        raise Exception ("Error in Parser")
                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")


    def printStatement(self, tree):
        nTree = NTerminalNode("PRINTSTATEMENT")

        if self.lexer.tokenList[0][0] == "__PRINT":
            self.getNextToken()
            # nTree.children.append(TerminalNode("__PRINT", self.lexer.tokenList[0][1])); self.getNextToken()
            
            self.expr(nTree)
            tree.children.append(nTree)
        
        else:
            raise Exception ("Error in Parser")


    def delayStatement(self, tree):
        nTree = NTerminalNode("DELAYSTATEMENT")

        if self.lexer.tokenList[0][0] == "__DELAY":
            self.getNextToken()
            # nTree.children.append(TerminalNode("__DELAY", self.lexer.tokenList[0][1])); self.getNextToken()
            
            self.expr(nTree)
            tree.children.append(nTree)

        else:
            raise Exception ("Error in Parser")


    def pixelStatement(self, tree):
        nTree = NTerminalNode("PIXELSTATEMENT")

        if self.lexer.tokenList[0][0] == "__PIXELR":
            nTree.children.append(TerminalNode("__PIXELR", self.lexer.tokenList[0][1])); self.getNextToken()
            self.expr(nTree)

            if self.lexer.tokenList[0][0] == ",":
                self.getNextToken()
                self.expr(nTree)

                if self.lexer.tokenList[0][0] == ",":
                    self.getNextToken()
                    self.expr(nTree)

                    if self.lexer.tokenList[0][0] == ",":
                        self.getNextToken()
                        self.expr(nTree)

                        if self.lexer.tokenList[0][0] == ",":
                            self.getNextToken()
                            self.expr(nTree)

                            tree.children.append(nTree)
                        
                        else:
                            raise Exception ("Error in Parser")
                    else:
                        raise Exception ("Error in Parser")
                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        
        elif self.lexer.tokenList[0][0] == "__PIXEL":
            nTree.children.append(TerminalNode("__PIXEL", self.lexer.tokenList[0][1])); self.getNextToken()
            self.expr(nTree)

            if self.lexer.tokenList[0][0] == ",":
                self.getNextToken()
                self.expr(nTree)

                if self.lexer.tokenList[0][0] == ",":
                    self.getNextToken()
                    self.expr(nTree)

                    tree.children.append(nTree)
                
                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")


    def rtrnStatement(self, tree):
        nTree = NTerminalNode("RTRNSTATEMENT")

        if self.lexer.tokenList[0][0] == "RETURN":
            self.getNextToken()
            # nTree.children.append(TerminalNode("RETURN", self.lexer.tokenList[0][1])); self.getNextToken()
            self.expr(nTree)

            tree.children.append(nTree)


    def ifStatement(self, tree):
        nTree = NTerminalNode("IFSTATEMENT")

        if self.lexer.tokenList[0][0] == "IF":
            self.getNextToken()
            # nTree.children.append(TerminalNode("IF", self.lexer.tokenList[0][1])); self.getNextToken()

            if self.lexer.tokenList[0][0] == "(":
                self.getNextToken()
                self.expr(nTree)
                
                if self.lexer.tokenList[0][0] == ")":
                    self.getNextToken()
                    self.block(nTree)
                    
                    if self.lexer.tokenList and self.lexer.tokenList[0][0] == "ELSE":
                        self.getNextToken()
                        # nTree.children.append(TerminalNode("ELSE", self.lexer.tokenList[0][1])); self.getNextToken()

                        self.block(nTree)

                        tree.children.append(nTree)

                    else:
                        tree.children.append(nTree)
                
                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")                


    def forStatement(self, tree):
        nTree = NTerminalNode("FORSTATEMENT")

        if self.lexer.tokenList[0][0] == "FOR":
            self.getNextToken()
            # nTree.children.append(TerminalNode("FOR", self.lexer.tokenList[0][1])); self.getNextToken()

            if self.lexer.tokenList[0][0] == "(":
                self.getNextToken()

                if self.lexer.tokenList[0][0] == "LET":
                    self.variableDecl(nTree)

                    if self.lexer.tokenList[0][0] == ";":
                        self.getNextToken()
                        self.expr(nTree)
                        self.getNextToken()

                        if self.lexer.tokenList[0][0] == "IDENTIFIER":
                            self.assignment(nTree)
                            
                            if self.lexer.tokenList[0][0] == ")":
                                self.getNextToken()
                                self.block(nTree)
                                tree.children.append(nTree)
                            
                            else:
                                raise Exception ("Error in Parser")
                            
                        elif self.lexer.tokenList[0][0] == ")":
                            self.getNextToken()
                            self.block(nTree)
                            tree.children.append(nTree)
                        
                        else:
                            raise Exception ("Error in Parser")
                    else:
                        raise Exception ("Error in Parser")
                        
                elif self.lexer.tokenList[0][0] == ";":
                    self.getNextToken()
                    self.expr(nTree)
                    self.getNextToken()

                    if self.lexer.tokenList[0][0] == "IDENTIFIER":
                        self.assignment(nTree)
                        
                        if self.lexer.tokenList[0][0] == ")":
                            self.getNextToken()
                            self.block(nTree)
                            tree.children.append(nTree)
                        
                        else:
                            raise Exception ("Error in Parser")
                        
                    elif self.lexer.tokenList[0][0] == ")":
                        self.getNextToken()
                        self.block(nTree)
                        tree.children.append(nTree)
                    
                    else:
                        raise Exception ("Error in Parser")
                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")


    def whileStatement(self, tree):
        nTree = NTerminalNode("WHILESTATEMENT")

        if self.lexer.tokenList[0][0] == "WHILE":
            self.getNextToken()
            # nTree.children.append(TerminalNode("WHILE", self.lexer.tokenList[0][1])); self.getNextToken()

            if self.lexer.tokenList[0][0] == "(":
                self.getNextToken()
                self.expr(nTree)

                if self.lexer.tokenList[0][0] == ")":
                    self.getNextToken()
                    self.block(nTree)
                    tree.children.append(nTree)
                
                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")


    def formalParam(self, tree):
        nTree = NTerminalNode("FORMALPARAM")

        if self.lexer.tokenList[0][0] == "IDENTIFIER":
            nTree.children.append(TerminalNode("IDENTIFIER", self.lexer.tokenList[0][1])); self.getNextToken()

            if self.lexer.tokenList[0][0] == ":":
                self.getNextToken()

                if self.lexer.tokenList[0][0] == "TYPE":
                    nTree.children.append(TerminalNode("TYPE", self.lexer.tokenList[0][1])); self.getNextToken()
                    
                    tree.children.append(nTree)
                
                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")

   
    def formalParams(self, tree):
        nTree = NTerminalNode("FORMALPARAMS")

        self.formalParam(nTree)

        while self.lexer.tokenList and self.lexer.tokenList[0][0] == ",":
            self.getNextToken()
            self.formalParam(nTree)

        tree.children.append(nTree)

   
    def functionDecl(self,tree):
        nTree = NTerminalNode("FUNCTIONDECL")

        if self.lexer.tokenList[0][0] == "FUN":
            self.getNextToken()
            # nTree.children.append(TerminalNode("FUN", self.lexer.tokenList[0][1])); self.getNextToken()
            
            if self.lexer.tokenList[0][0] == "IDENTIFIER":
                nTree.children.append(TerminalNode("IDENTIFIER", self.lexer.tokenList[0][1])); self.getNextToken()

                if self.lexer.tokenList[0][0] == "(":
                    self.getNextToken()

                    if self.lexer.tokenList[0][0] == "IDENTIFIER":
                        self.formalParams(nTree)

                        if self.lexer.tokenList[0][0] == ")":
                            self.getNextToken()
                             
                            if self.lexer.tokenList[0][0] == "->":
                                nTree.children.append(TerminalNode("ARROW", self.lexer.tokenList[0][1])); self.getNextToken()

                                if self.lexer.tokenList[0][0] == "TYPE":
                                    nTree.children.append(TerminalNode("TYPE", self.lexer.tokenList[0][1])); self.getNextToken()
                                    self.block(nTree)
                                    tree.children.append(nTree)
                                
                                else:
                                    raise Exception ("Error in Parser")
                            else:
                                raise Exception ("Error in Parser")
                        else:
                            raise Exception ("Error in Parser")
                                
                    elif self.lexer.tokenList[0][0] == ")":
                        self.getNextToken()
                            
                        if self.lexer.tokenList[0][0] == "->":
                            nTree.children.append(TerminalNode("ARROW", self.lexer.tokenList[0][1])); self.getNextToken()

                            if self.lexer.tokenList[0][0] == "TYPE":
                                nTree.children.append(TerminalNode("TYPE", self.lexer.tokenList[0][1])); self.getNextToken()
                                self.block(nTree)

                                tree.children.append(nTree)
                            
                            else:
                                raise Exception ("Error in Parser")
                        else:
                            raise Exception ("Error in Parser")
                    else:
                        raise Exception ("Error in Parser")
                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")
                            
                            
    def statement(self, tree): 
        nTree = NTerminalNode("STATEMENT")

        # VARIABLEDECL
        if self.lexer.tokenList[0][0] == "LET":
            self.variableDecl(nTree)

            if self.lexer.tokenList[0][0] == ";":
                self.getNextToken()
            else:
                raise Exception ("Error: missing semi-colon")

        # ASSIGNMENT
        elif self.lexer.tokenList[0][0] == "IDENTIFIER":
            self.assignment(nTree)
            
            if self.lexer.tokenList and self.lexer.tokenList[0][0] == ";":
                self.getNextToken()

            else:
                raise Exception ("Error: missing semi-colon")

        # PRINTSTATEMENT
        elif self.lexer.tokenList[0][0] == "__PRINT":
            self.printStatement(nTree)

            if self.lexer.tokenList[0][0] == ";":
                self.getNextToken()

            else:
                raise Exception ("Error: missing semi-colon")

        # DELAYSTATEMENT
        elif self.lexer.tokenList[0][0] == "__DELAY":
            self.delayStatement(nTree)

            if self.lexer.tokenList[0][0] == ";":
                self.getNextToken()

            else:
                raise Exception ("Error: missing semi-colon")

        # PIXELSTATEMENT
        elif self.lexer.tokenList[0][0] in ["__PIXEL", "__PIXELR"]:
            self.pixelStatement(nTree)

            if self.lexer.tokenList[0][0] == ";":
                self.getNextToken()

            else:
                raise Exception ("Error: missing semi-colon")

        # IFSTATEMENT
        elif self.lexer.tokenList[0][0] == "IF":
            self.ifStatement(nTree)

        # FORSTATEMENT
        elif self.lexer.tokenList[0][0] == "FOR":
            self.forStatement(nTree)

        # WHILESTATEMENT
        elif self.lexer.tokenList[0][0] == "WHILE":
            self.whileStatement(nTree)

        # RETURN
        elif self.lexer.tokenList[0][0] == "RETURN":
            self.rtrnStatement(nTree)
            
            if self.lexer.tokenList[0][0] == ";":
                self.getNextToken()

            else:
                raise Exception ("Error: missing semi-colon")

        # FUNCTIONDECL
        elif self.lexer.tokenList[0][0] == "FUN":
            self.functionDecl(nTree)

        # BLOCK
        elif self.lexer.tokenList[0][0] == "{":
            self.block(nTree)
            
        else:
            raise Exception ("Error: invalid STATEMENT")
        
        tree.children.append(nTree)

   
    def block(self, tree):
        nTree = NTerminalNode("BLOCK")

        if self.lexer.tokenList[0][0] == "{":
            self.getNextToken()
            
            if self.lexer.tokenList[0][0] != "}":
                
                while self.lexer.tokenList[0][0] != "}":
                    (self.statement(nTree))

                self.getNextToken()
            elif self.lexer.tokenList[0][0] == "}":
                self.getNextToken()
            else:
                raise Exception ("Error: missing '}' to close BLOCK")
            tree.children.append(nTree)
        else:
            raise Exception ("Error: missing '{' to start BLOCK")

   
    def program(self):
        tree = NTerminalNode("PROGRAM")
        
        while self.lexer.tokenList:
            (self.statement(tree))

        self.tree = tree

In [117]:
f = open('code.txt', 'r'); code = f.read(); f.close()
parser = Parser(code)

L E X E R
Code accepted by Lexer

P A R S E R
Code Accepted by Parser


# Task 3 - AST XML Generation Pass

In [118]:
class XMLGeneration:
    def __init__(self, code):
        parser = Parser(code)
        self.AST = parser.tree
        self.stack = []
        self.space = ""

        self.generateXML()


    def generateXML(self):
        print("\nX M L   G E N E R A T I O N")
        print("Full AST:")
        self.printAST(self.AST)
        print("\nMinimized AST:")
        self.printASTMinimized(self.AST)


    def printAST(self, tree):
        if str(type(tree)) == "<class '__main__.NTerminalNode'>":
            print(self.space + "<" + tree.name + ">")

            self.stack.append(tree.name); self.space += "  "

            for child in tree.children:
                self.printAST(child)
            
            self.space = self.space[:-2]
            print(self.space + "</" + self.stack[-1] + ">")
            self.stack.pop(); 
        else:
            print(self.space + "<" + str(tree.name) + "> " + str(tree.value) + " </" + tree.name + ">")


    def printASTMinimized(self, tree):
        if str(type(tree)) == "<class '__main__.NTerminalNode'>":

            if len(tree.children) != 1 or tree.name == "PROGRAM":
                print(self.space + "<" + tree.name + ">")
                self.stack.append(tree.name); self.space += "  "

                for child in tree.children:
                    self.printASTMinimized(child)
                
                self.space = self.space[:-2]
                print(self.space + "</" + self.stack[-1] + ">")
                self.stack.pop(); 
            else:
                for child in tree.children:
                    self.printASTMinimized(child)        
                    
        else:
            print(self.space + "<" + str(tree.name) + "> " + str(tree.value) + " </" + tree.name + ">")

In [119]:
f = open('code.txt', 'r'); code = f.read(); f.close()

xml = XMLGeneration(code)

L E X E R
Code accepted by Lexer

P A R S E R
Code Accepted by Parser

X M L   G E N E R A T I O N
Full AST:
<PROGRAM>
  <STATEMENT>
    <FUNCTIONDECL>
      <IDENTIFIER> MoreThan50 </IDENTIFIER>
      <FORMALPARAMS>
        <FORMALPARAM>
          <IDENTIFIER> x </IDENTIFIER>
          <TYPE> int </TYPE>
        </FORMALPARAM>
      </FORMALPARAMS>
      <ARROW> -> </ARROW>
      <TYPE> bool </TYPE>
      <BLOCK>
        <STATEMENT>
          <VARIABLEDECL>
            <IDENTIFIER> y </IDENTIFIER>
            <TYPE> int </TYPE>
            <EQUALS> = </EQUALS>
            <EXPR>
              <SIMPLEEXPR>
                <TERM>
                  <FACTOR>
                    <LITERAL>
                      <INTEGERLITERAL> 23 </INTEGERLITERAL>
                    </LITERAL>
                  </FACTOR>
                </TERM>
              </SIMPLEEXPR>
            </EXPR>
          </VARIABLEDECL>
        </STATEMENT>
        <STATEMENT>
          <IFSTATEMENT>
            <EXPR>
     

# Task 4 - Semantic Analysis Pass


In [130]:
# THINGS DONE
# VARIABLEDECL expression must be same type as TYPE
# VARIABLEDECL must not declare variable with same name as those in the current (or higher) scopes

# ASSIGNMENT must use a pre existing variable
# ASSIGNMENT's value type must match the type in the variable's decleration

# if FUNCTIONDECL has a return statement (or more than 1), the return values must match the function type 

# __PIXEL must have integer, integer, colour
# __PIXELR must have integer, integer, integer, integer, colour

class symbolTable:
    def __init__(self):
        self.variables = []
        self.functions = []

class semanticAnalysis:
    def __init__(self, tree):
        self.symbols = symbolTable()
        self.tree = tree
        self.counter = 0

        print("\nS E M A N T I C   A N A L Y S I S")
        self.traverse(self.tree)
        print("Code passed Semantic Analysis")


    def printDetails(self, tree, slash):

        if slash == False: print (tree.name)
        else: print("/" + tree.name)
        
        print("Variables: " + str(self.symbols.variables))
        print("Functions: " + str(self.symbols.functions))
        print("Num of Children: " + str(len(tree.children)))
        print()


    def traverse(self, tree):
        # self.printDetails(tree, False)

        if tree.name == "PROGRAM":
            self.addScopePass(tree, None)

        if tree.name == "BLOCK":
            self.addScopePass(tree, None)

        if tree.name == "STATEMENT":
            self.traverse(tree.children[0])

        if tree.name == "VARIABLEDECL":
            vName = tree.children[0].value
            vType = tree.children[1].value

            for scope in self.symbols.variables:
                if vName in scope:
                    raise Exception ("Error: variable '" + vName + "' already declared")
                
            vValue = tree.children[3]
            vValueType = self.getVarType(vValue)

            if vType != vValueType:
                raise Exception ("Error: variable '" + vName + "' does not accept type '" + str(vValueType) + "'")
                
            self.symbols.variables[-1][vName] = vType

        if tree.name == "ASSIGNMENT":
            vName = tree.children[0].value
            
            flag = False
            for scope in self.symbols.variables:
                if vName in scope:
                    flag = True
                    break

            if flag == False:
                raise Exception ("Error: variable '" + vName + "' not defined")

            vValue = tree.children[2]
            vType = self.getVarType(vValue)

            reqType = ""
            for scope in self.symbols.variables:
                if vName in scope:
                    reqType = scope[vName]

            if vType != reqType:
                raise Exception ("Error: variable '" + vName + "' does not accept type '" + vType + "'")
            
        if tree.name == "PIXELSTATEMENT":
            
            if tree.children[0].name == "__PIXEL":
                a = self.getVarType(tree.children[1])
                b = self.getVarType(tree.children[2])
                c = self.getVarType(tree.children[3])

                if a == "int" and b == "int" and c == "colour":
                    pass
                else:
                    raise Exception ("Error: invalid format for __PIXEL (int, int, colour)")
            else:
                a = self.getVarType(tree.children[1])
                b = self.getVarType(tree.children[2])
                c = self.getVarType(tree.children[3])
                d = self.getVarType(tree.children[4])
                e = self.getVarType(tree.children[5])

                if a == "int" and b == "int" and c == "int" and d == "int" and e == "colour":
                    pass
                else:
                    raise Exception ("Error: invalid format for __PIXELR (int, int, int, int, colour)")
            
        if tree.name == "FUNCTIONDECL":
            funName = tree.children[0].value
            funType = ""
            

            # No Parameters
            if tree.children[3].name == "ARROW":
                funType = tree.children[4].value
                

                for scope in self.symbols.functions:
                    if funName in scope:
                        raise Exception ("Error: function '" + funName + "' already declared")

                listOfReturns = self.getReturns(tree)

                temp = []
                for x in listOfReturns:
                    temp.append(self.getVarType(x.children[0]))
                    
                flag = True
                for x in temp:
                    if x != funType:
                        flag = False

                if flag == False:
                        raise Exception ("Error: Return values dont match function type")
                
                
                self.symbols.functions[-1][funName] = [funType, None]
                self.addScopePass(tree.children[4], None)
                
            # Yes Parameters
            else:
                funType = tree.children[3].value

                for scope in self.symbols.functions:
                    if funName in scope:
                        raise Exception ("Error: function '" + funName + "' already declared")

                list = []

                for param in tree.children[1].children:
                    pName = param.children[0].value

                    for scope in self.symbols.variables:
                        if pName in scope:
                            raise Exception ("Error: variable '" + pName + "' already declared")
                    
                    pType = param.children[1].value
                    list.append((pName, pType))

                    listOfReturns = self.getReturns(tree)
                    
                    temp = []
                    for x in listOfReturns:
                        temp.append(self.getVarType(x.children[0]))
                        
                    flag = True
                    for x in temp:
                        if x != funType:
                            flag = False

                    if flag == False:
                        raise Exception ("Error: Return values dont match function type")
                    
                
                parameterTypes = []
                for i in list:
                    parameterTypes.append(i[1])

                self.symbols.functions[-1][funName] = [funType, parameterTypes]
                self.addScopePass(tree.children[4], list)

        if tree.name == "IFSTATEMENT":
            self.getVarType(tree.children[0])

        if tree.name == "WHILESTATEMENT":
            for i in tree.children:
                self.traverse(i)

            # self.getVarType(tree.children[0])

        if tree.name == "FORSTATEMENT":
            for i in tree.children:
                self.traverse(i)

        # self.printDetails(tree, True)
           


    # function to create a new scope
    def addScopePass(self, tree, list):
        self.symbols.variables.append({}); self.symbols.functions.append({})

        if list != None:
            for pair in list:
                self.symbols.variables[-1][pair[0]] = pair[1]

        for child in tree.children:
                self.traverse(child)
        self.symbols.variables.pop(-1); self.symbols.functions.pop(-1)
    
    # functions to get value type of expression
    def getVarType(self, tree):        

        if str(type(tree)) == "<class '__main__.NTerminalNode'>":

            if tree.name == "FUNCTIONCALL":
                funName = tree.children[0].value
                reqParams = ""
                for scope in self.symbols.functions:
                    if funName in scope:
                        reqParams = scope[funName][1]

                if reqParams == "":
                    raise Exception ("Error: function '"+ funName + "' isnt declared")
                
                if reqParams == None:
                    if len(tree.children) != 1:
                        raise Exception ("Error: function '" + funName + "' doesnt take paramters")

                else:
                    if len(tree.children[1].children) != len(reqParams):
                        raise Exception ("Error: more values than expected")

                    tempList = []
                    for i in tree.children[1].children:
                        tempList.append(self.getVarType(i))

                    for counter in range(len(tempList)):
                        if tempList[counter] != reqParams[counter]:
                            raise Exception ("Error: invalid paramters passed")
                        
                for scope in self.symbols.functions:
                    if funName in scope:
                        return scope[funName][0]

            else:                    
                types = []
                for child in tree.children:
                    varType = self.getVarType(child)

                    if varType != None:
                        types.append(varType)

                if len(types) == 1:
                    return types[0]
                else:
                    return self.checkTypes(types)

        else:
            if tree.name == "INTEGERLITERAL" or tree.name == "PADWIDTH" or tree.name == "PADHEIGHT":
                return ("int")
            
            elif tree.name == "FLOATLITERAL":
                return ("float")
            
            elif tree.name == "BOOLEANLITERAL":
                return ("bool")
            
            elif tree.name == "COLOURLITERAL":
                return ("colour")
            
            elif tree.name == "IDENTIFIER":
                name = tree.value
                reqType = ""

                for scope in self.symbols.variables:
                    if name in scope:
                        reqType = scope[name]

                if reqType == "":
                    raise Exception ("Error: variable '" + name + "' does not exist")

                return reqType

    def checkTypes(self, list):
        if len(list) == 0:
            return None

        elif len(list) == 1:           
            return list[0]

        elif len(list) > 2:
            while len(list) > 2:
                tempList = [list.pop(0), list.pop(0)]
                newVal = self.checkTypes(tempList)
                list.append(newVal)
            
        else:
            if list[0] == "int" and list[1] == "int":
                return "int"
            elif list[0] == "float" and list[1] == "int" or list[0] == "int" and list[1] == "float" or list[0] == "float" and list[1] == "float":
                return "float"
            elif list[0] == "colour" and list[1] == "colour":
                return "colour"
            elif list[0] == "bool" and list[1] == "bool":
                return "bool"
            else:
                raise Exception ("Error in type matching")
            
    # function to get each return type in a function decleration
    def getReturns(self, tree):
        num = []
        if tree.name == "RTRNSTATEMENT":
            num.append(tree)
            return num
        
        elif str(type(tree)) == "<class '__main__.NTerminalNode'>":

            for child in tree.children:
                temp = self.getReturns(child)
                
                for i in temp:
                    num.append(i)

        return num


In [131]:
f = open('code.txt', 'r'); code = f.read(); f.close()

parser = Parser(code)

semAnalysis = semanticAnalysis(parser.tree)

L E X E R
Code accepted by Lexer

P A R S E R
Code Accepted by Parser

S E M A N T I C   A N A L Y S I S
Code passed Semantic Analysis


# Task 5 - PixIR Code Generation Pass
